In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
from IPython.display import display
import pprint

import codecs
import json

from xmlreader import XmlReader
from audit import KeyAuditor, StreetAuditor

SAMPLE_SKIP = 50
DATA_FILE = 'data/buenos-aires_argentina.osm'
DATA_SAMPLE_FILE = 'data/buenos-aires_argentina_sample_%d.osm' % SAMPLE_SKIP

In [41]:
data = XmlReader(DATA_FILE)
#display(data.count_tags(filter_tag='node'))

In [42]:
#todo:
# 1. recode method to shrink XML file -> done
data.write_sample_file(n_skip=SAMPLE_SKIP)
# 2. create small XML file to test tags -> done
# 3. move XmlReader and KeyAuditor classes to module file -> done
# 4. finish streets audit and write down encountered problems

In [43]:
datasample = XmlReader(DATA_SAMPLE_FILE)

In [106]:
display(dict(datasample.count_tags()))

Holaaaa


In [ ]:
key_auditor = KeyAuditor(datasample)
display(dict(key_auditor.test(filter_tags=('tag'))))

In [ ]:
"""
Your task is to explore the data a bit more.
The first task is a fun one - find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")
"""

def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(element.attrib['uid'])

    return users


def test3():

    users = process_map(DATA_FILE)
    display(len(users))
    # assert len(users) == 6

Identifying street types is not as easy as doing it with English:
* Street names don't include the word *Calle* explicitly, but avenues, boulevards, etc do. This means that any *way* not specifying the type on its name can be considered a normal street.
* Some localities use number and/or names for streets

In [233]:
street_auditor = StreetAuditor(data)
street_types = street_auditor.audit_types(unknown_ignore=True)

In [235]:
display(street_types['types'].keys())
display(street_types['types']['Unknown'])

dict_keys(['Calle', 'Unknown', 'Cno', 'Ave', 'Av', 'Línea', 'Pje', 'Bv'])

{'- Maipú',
 '1 Bis',
 '1 e/76bis y 77 nro 2222',
 '1 esq 65 nro 1599',
 '1 y Diagonal 80 № 605',
 '10 y 479',
 '103 - 26 de Julio de 1890',
 '106 Leandro N. Alem',
 '107 Patagonia',
 '109 - 1º de Agosto de 1806',
 '11 e/ 33 y 34 nro 77',
 '11 e/ 490 y 491',
 '11 esq 46 nro 705',
 '11 y 530',
 '110 José Hernández',
 '110 Pueyrredón',
 '113 Andrés Folch',
 '114 Alvear',
 '114 Alvear`1',
 '115 esq Diagonal 74 № 234',
 '116 esq 37 nro 251',
 '116 esq. 522',
 '12 Bis',
 '12 entre 67 y 68',
 '12 esq 62 nro 1449',
 '12 esq. 44 nro 595',
 '120 e/72 y 73 nro 1980',
 '120 nro 2181 e/ 76 y 77',
 '122 General Paz',
 '122 e/43 y 44 nro 598',
 '122 e/62 y 63 nro 1464',
 '122 esq 70',
 '122 esq 90',
 '122 nro  2151 esq 76',
 '126 y 43',
 '13 e/ 528 y 529 nro 333',
 '13 e/33 y 34 nro 83',
 '13 esq 77 nro 2198',
 '131 esq 40 nro 399',
 '133 Bis',
 '134 Bis',
 '136 Bis',
 '137 Bis',
 '137 e/43 y 44 nro 586',
 '138 Bis',
 '14 esq 66 nro 1651',
 '14 y 466',
 '140 Profesor Guillermo Simón',
 '141 Bis',
 '

In [5]:
"""
Your task is to wrangle the data and transform the shape of the data
into the model we mentioned earlier. The output should be a list of dictionaries
that look like this:

{
"id": "2406124091",
"type: "node",
"visible":"true",
"created": {
          "version":"2",
          "changeset":"17206049",
          "timestamp":"2013-08-03T16:43:42Z",
          "user":"linuxUser16",
          "uid":"1219059"
        },
"pos": [41.9757030, -87.6921867],
"address": {
          "housenumber": "5157",
          "postcode": "60625",
          "street": "North Lincoln Ave"
        },
"amenity": "restaurant",
"cuisine": "mexican",
"name": "La Cabana De Don Luis",
"phone": "1 (773)-271-5176"
}

You have to complete the function 'shape_element'.
We have provided a function that will parse the map file, and call the function with the element
as an argument. You should return a dictionary, containing the shaped data for that element.
We have also provided a way to save the data in a file, so that you could use
mongoimport later on to import the shaped data into MongoDB. 

Note that in this exercise we do not use the 'update street name' procedures
you worked on in the previous exercise. If you are using this code in your final
project, you are strongly encouraged to use the code from previous exercise to 
update the street names before you save them to JSON. 

In particular the following things should be done:
- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
- if the second level tag "k" value contains problematic characters, it should be ignored
- if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if the second level tag "k" value does not start with "addr:", but contains ":", you can
  process it in a way that you feel is best. For example, you might split it into a two-level
  dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:

<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>

  should be turned into:

{...
"address": {
    "housenumber": 5158,
    "street": "North Lincoln Avenue"
}
"amenity": "pharmacy",
...
}

- for "way" specifically:

  <nd ref="305896090"/>
  <nd ref="1719825889"/>

should be turned into
"node_refs": ["305896090", "1719825889"]
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way":
        node['type'] = element.tag
        for a in element.attrib:
            #print('iterating node/way:', tag, tag.attrib)
            if a in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][a] = element.attrib[a]
            elif a in ('lat','lon'):
                if 'pos' not in node:
                    node['pos'] = [0,0]
                key = 0 if a == 'lat' else 1
                node['pos'][key] = float(element.attrib[a])
            else:
                node[a] = element.attrib[a]
        for t in element.iter('tag'):
            if problemchars.search(t.attrib['k']):
                continue
            elif t.attrib['k'].startswith('addr:'):
                if 'address' not in node:
                    node['address'] = {}
                if t.attrib['k'].count(':') == 1:
                    addr_type = t.attrib['k'].split(':')[1]
                    node['address'][addr_type] = t.attrib['v']
            elif t.attrib['k'].count(':') == 1:
                extra_node = t.attrib['k'].split(':')
                if extra_node[0] not in node:
                    node[extra_node[0]] = {}
                node[extra_node[0]][extra_node[1]] = t.attrib['v']
        if element.tag == 'way':
            for t in element.iter('nd'):
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in t.attrib:
                    node['node_refs'].append(t.attrib['ref'])
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test5():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map(DATA_FILE, True)
    pprint.pprint(data[-1])
    
    """correct_first_elem = {
        "id": "261114295", 
        "visible": "true", 
        "type": "node", 
        "pos": [41.9730791, -87.6866303], 
        "created": {
            "changeset": "11129782", 
            "user": "bbmiller", 
            "version": "7", 
            "uid": "451048", 
            "timestamp": "2012-03-28T18:31:23Z"
        }
    }
    assert data[0] == correct_first_elem
    assert data[-1]["address"] == {
                                    "street": "West Lexington St.", 
                                    "housenumber": "1412"
                                      }
    assert data[-1]["node_refs"] == [ "2199822281", "2199822390",  "2199822392", "2199822369", 
                                    "2199822370", "2199822284", "2199822281"]"""

NameError: name 're' is not defined

In [ ]:
test5()

In [ ]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "some_osm.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')